In [67]:
import gdown
import pandas
import os
import numpy
from tabulate import tabulate
from sklearn.cluster import MeanShift, SpectralClustering, estimate_bandwidth
from sklearn.metrics import silhouette_score, davies_bouldin_score

file_id = "1TMto04p1s_ntL9PZdygdV22S_RjAqLiy"

temp_dir = 'temp_folder'
os.makedirs(temp_dir, exist_ok=True)

try:
    gdown.download(id=file_id, output=temp_dir+'/data.csv', quiet=False)
except Exception as e:
    print(f"Error downloading file: {e}")

data = pandas.read_csv(temp_dir+'/data.csv')

#find missing data
missing_data = data.isna().sum()
row_count = len(data)
percent_missing = (missing_data / row_count)*100

print(" ==== Percent Missing per Row ==== ")
print(percent_missing)

#fix data
data = data.dropna()

row_count = len(data)

data['ALARM DATE TIME'] = pandas.to_datetime(data['ALARM DATE TIME'], format='%m/%d/%y %H:%M')
data['CALL COMPLETE'] = pandas.to_datetime(data['CALL COMPLETE'], format='%m/%d/%y %H:%M')

data['CALL COMPLETE'] = data['ALARM DATE TIME'].dt.date.astype(str) + ' ' + data['CALL COMPLETE'].dt.time.astype(str)
data['CALL COMPLETE'] = pandas.to_datetime(data['CALL COMPLETE'], format='%Y-%m-%d %H:%M:%S')

data['DISPATCH UNIT'] = data['DISPATCH UNIT'].str.replace('[A-Z]', '', regex=True)
data['1ST UNIT ON SCENE'] = data['1ST UNIT ON SCENE'].str.replace('[A-Z]', '', regex=True)

#Average Response Time
average_diff = ((data['CALL COMPLETE'] - data['ALARM DATE TIME']).sum() / row_count)
print()
print(" ==== Average Response Time ==== ")
print(average_diff)

#Average Units Dispatched
total_dispatch = (data['DISPATCH UNIT'].str.split(',').str.len()).sum() / row_count
print()
print(" ==== Average Units Dispatched ==== ")
print(total_dispatch)

#Busiest Shift
percent_a = ((data['SHIFT'] == "A").sum() / row_count) * 100
percent_b = ((data['SHIFT'] == "B").sum() / row_count) * 100
percent_c = ((data['SHIFT'] == "C").sum() / row_count) * 100
print()
print(" ==== Busiest Shift ==== ")
print("Shift A:",percent_a)
print("Shift B:",percent_b)
print("Shift C:",percent_c)

#Day/Hour Matrix
data['DayOfWeek'] = data['ALARM DATE TIME'].dt.dayofweek  # 0=Monday
data['HourOfDay'] = data['ALARM DATE TIME'].dt.hour

day_order = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
data['DayName'] = data['DayOfWeek'].map({0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})

matrix = pandas.pivot_table(
    data,
    index='HourOfDay',
    columns='DayName',
    values='ALARM DATE TIME',
    aggfunc='count',
    fill_value=0
)

matrix = matrix.reindex(columns=day_order, fill_value=0)

matrix['Row Total'] = matrix.sum(axis=1)
totals_row = matrix.sum(axis=0).to_frame().T
totals_row.index = ['Column Total']
matrix = pandas.concat([matrix, totals_row])

print()
print(" ==== Day/Hour Matrix ==== ")
print(tabulate(matrix, headers='keys', tablefmt='grid'))

#Sci-Kit Cluster
X = matrix.drop(columns=['Row Total', 'Column Total'], errors='ignore')

ms_labels = MeanShift(bandwidth=estimate_bandwidth(X)).fit_predict(X)
ms_sil = silhouette_score(X, ms_labels)
ms_db = davies_bouldin_score(X, ms_labels)

spectral_labels = SpectralClustering(n_clusters=3).fit_predict(X)
spectral_sil = silhouette_score(X, spectral_labels)
spectral_db = davies_bouldin_score(X, spectral_labels)

print()
print("==== Sci-kit Clusters ====")
print("Mean Shift: Silhouette Score =", ms_sil, "Davies-Bouldin Index =", ms_db)
print("Spectral Clustering: Silhouette Score =", spectral_sil, "Davies-Bouldin Index =", spectral_db)

if os.path.exists(temp_dir):
        for f in os.listdir(temp_dir):
            os.remove(os.path.join(temp_dir, f))
        os.rmdir(temp_dir)

Downloading...
From: https://drive.google.com/uc?id=1TMto04p1s_ntL9PZdygdV22S_RjAqLiy
To: /home/bdshilli/Development/Github/CSCE580/Quiz1/temp_folder/data.csv
100%|██████████| 167k/167k [00:00<00:00, 2.34MB/s]

 ==== Percent Missing per Row ==== 
XREF ID                   0.000000
DISPATCH UNIT             0.000000
DISPATCH CREATED DATE     0.000000
INCIDENT NUMBER           0.000000
1ST UNIT ON SCENE        19.454545
ALARM DATE TIME           1.409091
CALL COMPLETE             1.409091
SHIFT                     3.136364
dtype: float64

 ==== Average Response Time ==== 
0 days 00:08:23.127906976

 ==== Average Units Dispatched ==== 
1.4779069767441861

 ==== Busiest Shift ==== 
Shift A: 34.47674418604651
Shift B: 32.383720930232556
Shift C: 33.13953488372093

 ==== Day/Hour Matrix ==== 
+--------------+----------+----------+-----------+-------------+------------+----------+------------+-------------+
|              |   Sunday |   Monday |   Tuesday |   Wednesday |   Thursday |   Friday |   Saturday |   Row Total |
+==============+==========+==========+===========+=============+============+==========+============+=============+
| 0            |        4 |        7 |         6 |           4 | 


/home/bdshilli/Development/Github/CSCE580/.venv/lib/python3.12/site-packages/sklearn/manifold/_spectral_embedding.py:328: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/home/bdshilli/Development/Github/CSCE580/.venv/lib/python3.12/site-packages/sklearn/base.py:1365: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Q3 Text Answers

a1: Dispatches between 3/24/25 and 8/31/25

a2: ==== Percent Missing per Row ==== 
    XREF ID                   0%
    DISPATCH UNIT             0%
    DISPATCH CREATED DATE     0%
    INCIDENT NUMBER           0%
    1ST UNIT ON SCENE        19.45%
    ALARM DATE TIME           1.41%
    CALL COMPLETE             1.41%
    SHIFT                     3.14%

a3: invalid date values - Replace CALL COMPLETE date with ALARM DATE TIME date
    Empty values - remove rows with missing values
    Format discrepency between 1ST UNIT ON SCENE and DISPATCH UNIT - Change dispatch references to use numbers only.

a4: Replace empty date time values with 10/10/10 10:10, remove all letters from DISPATCH UNIT and 1ST UNIT ON SCENE references

b1: On average, the reponse time from begining to finish is 8 minutes and 23 seconds

b2: On average, 1.48 units are dispatched

b3: Shift A is the busiest

c1: Mean Shift appears to preform better than Spectral Clustering

c2:

